In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cleaned-genre-df/sample_submission.csv
/kaggle/input/cleaned-genre-df/new_train.csv
/kaggle/input/cleaned-genre-df/new_test.csv
/kaggle/input/distilroberta/distilroberta_test.npy
/kaggle/input/distilroberta/distilroberta_df.npy
/kaggle/input/genre-df/sample_submission.csv
/kaggle/input/genre-df/train.csv
/kaggle/input/genre-df/test.csv


In [2]:
# Reading the df (data cleaning already done in task1_data_cleaning)
df = pd.read_csv('/kaggle/input/cleaned-genre-df/new_train.csv')
test = pd.read_csv('/kaggle/input/cleaned-genre-df/new_test.csv') 
sample_submission = pd.read_csv('/kaggle/input/cleaned-genre-df/sample_submission.csv')
classes_list = sample_submission.columns[1:]
print(len(classes_list))
print(df.shape)
print(test.shape)
print(sample_submission.shape)
df.head()

70
(3000, 5)
(864, 3)
(864, 71)


,index,plot_synopsis,tags,cleaned_plot_synopsis,cleaned_tags
0,0,Charlyne Yi embarks on a quest across America ...,"paranormal, psychedelic, storytelling, home movie",charlyn yi embark quest across america make do...,"['paranormal', 'psychedelic', 'storytelling', ..."
1,1,The movie is a parody of classic kung fu movie...,"cult, comedy, comic, violence, flashback",movi parodi classic kung fu movi featur footag...,"['cult', 'comedy', 'comic', 'violence', 'flash..."
2,2,"In 2011, João ""Zero"", a bitter but brilliant p...",romantic,jo zero bitter brilliant physicist spend day b...,['romantic']
3,3,Kent McCoy is a loving husband and father who ...,murder,kent mccoy love husband father host birthday p...,['murder']
4,4,The animated introduction revolves around how ...,"comedy, murder, violence, horror, flashback, s...",anim introduct revolv around devon voic snoop ...,"['comedy', 'murder', 'violence', 'horror', 'fl..."


In [3]:
import ast

# Assuming df is your DataFrame with the 'cleaned_tags' column containing strings representing lists

# Convert string representations of lists to actual lists
df['cleaned_tags'] = df['cleaned_tags'].apply(ast.literal_eval)


In [4]:
# Generated in task1_bert_gen

distilroberta_df = np.load('/kaggle/input/distilroberta/distilroberta_df.npy')
distilroberta_test = np.load('/kaggle/input/distilroberta/distilroberta_test.npy')

In [5]:
concatenated_df = pd.concat([df, pd.DataFrame(distilroberta_df )], axis=1)
concatenated_test = pd.concat([test, pd.DataFrame(distilroberta_test)], axis=1)

In [6]:
df

,index,plot_synopsis,tags,cleaned_plot_synopsis,cleaned_tags
0,0,Charlyne Yi embarks on a quest across America ...,"paranormal, psychedelic, storytelling, home movie",charlyn yi embark quest across america make do...,"[paranormal, psychedelic, storytelling, home_m..."
1,1,The movie is a parody of classic kung fu movie...,"cult, comedy, comic, violence, flashback",movi parodi classic kung fu movi featur footag...,"[cult, comedy, comic, violence, flashback]"
2,2,"In 2011, João ""Zero"", a bitter but brilliant p...",romantic,jo zero bitter brilliant physicist spend day b...,[romantic]
3,3,Kent McCoy is a loving husband and father who ...,murder,kent mccoy love husband father host birthday p...,[murder]
4,4,The animated introduction revolves around how ...,"comedy, murder, violence, horror, flashback, s...",anim introduct revolv around devon voic snoop ...,"[comedy, murder, violence, horror, flashback, ..."
...,...,...,...,...,...
2995,2995,A fictional pro football league finds themselv...,comedy,fiction pro footbal leagu find hit player stri...,[comedy]
2996,2996,The Prophet's Game involves a game master (the...,murder,prophet game involv game master killer give ob...,[murder]
2997,2997,When North Carolina secedes from the Union on ...,"depressing, murder, dramatic, flashback, roman...",north carolina seced union may young man rural...,"[depressing, murder, dramatic, flashback, roma..."
2998,2998,Roderick Blank (Simon Baker) is a successful y...,pornographic,roderick blank simon baker success young busin...,[pornographic]


In [7]:
df['cleaned_tags']

0       [paranormal, psychedelic, storytelling, home_m...
1              [cult, comedy, comic, violence, flashback]
2                                              [romantic]
3                                                [murder]
4       [comedy, murder, violence, horror, flashback, ...
                              ...                        
2995                                             [comedy]
2996                                             [murder]
2997    [depressing, murder, dramatic, flashback, roma...
2998                                       [pornographic]
2999                                         [paranormal]
Name: cleaned_tags, Length: 3000, dtype: object

In [8]:
print(df['cleaned_tags'][25][0])

comedy


In [9]:
# Target encoding
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer(classes=classes_list)
mlb.fit([classes_list])
print(mlb.classes_)
# transform target variable
y = mlb.transform(df['cleaned_tags'].values)

y_df = pd.DataFrame(y,columns=classes_list)
print(y.shape)
y

['revenge' 'boring' 'depressing' 'brainwashing' 'historical_fiction'
 'comic' 'psychological' 'neo_noir' 'plot_twist' 'sentimental' 'flashback'
 'storytelling' 'dark' 'thought-provoking' 'anti_war' 'action' 'cruelty'
 'christian_film' 'mystery' 'whimsical' 'avant_garde' 'haunting' 'cult'
 'pornographic' 'entertaining' 'historical' 'realism' 'violence' 'gothic'
 'home_movie' 'psychedelic' 'paranormal' 'queer' 'blaxploitation'
 'alternate_reality' 'comedy' 'magical_realism' 'philosophical' 'satire'
 'absurd' 'fantasy' 'bleak' 'alternate_history' 'clever' 'atmospheric'
 'melodrama' 'murder' 'inspiring' 'horror' 'suicidal' 'cute' 'stupid'
 'feel-good' 'sci-fi' 'allegory' 'good_versus_evil' 'romantic'
 'claustrophobic' 'humor' 'tragedy' 'sadist' 'western' 'adult_comedy'
 'dramatic' 'suspenseful' 'insanity' 'prank' 'intrigue' 'autobiographical'
 'grindhouse_film']
(3000, 70)


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [10]:
# # Train-test split
# # split dataset into training and validation set
# from sklearn.model_selection import train_test_split
# xtrain, xval, ytrain, yval = train_test_split(df['cleaned_plot_synopsis'], y, test_size=0.2)
X = concatenated_df.drop(['index','tags','cleaned_tags','plot_synopsis'],axis=1).values  # Convert to a numpy array with correct shape
y = y_df.values  # Convert to a numpy array

from skmultilearn.model_selection import iterative_train_test_split
xtrain, ytrain, xval, yval = iterative_train_test_split(X, y, test_size=0.2)

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Define TF-IDF vectorizer with additional parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200, norm='l2', sublinear_tf=True,
                                   stop_words='english', ngram_range=(1, 2))

# Transform train, validation, and test sets
xtrain_tfidf = tfidf_vectorizer.fit_transform(xtrain[:, 0])
xval_tfidf = tfidf_vectorizer.transform(xval[:, 0])
xtest_tfidf = tfidf_vectorizer.transform(test['cleaned_plot_synopsis'])


In [12]:
# Combining tf-idf and embeddings

xtrain_combined = np.hstack((xtrain_tfidf.toarray(), xtrain[:, 1:]))
xval_combined = np.hstack((xval_tfidf.toarray(),xval[:, 1:]))
xtest_combined = np.hstack((xtest_tfidf.toarray(), distilroberta_test))

print(xtrain_combined.shape)
print(ytrain.shape)
print(xtest_combined.shape)

(2429, 968)
(2429, 70)
(864, 968)


In [13]:
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import ParameterGrid

# Define the range of values for t and the variance ratio for PCA
t_values = [0.05, 0.1,0.5]
variance_ratios = [0.9, 0.95, 0.99]

best_f1_macro = 0
best_params = {}

# Perform grid search
for t in t_values:
    for variance_ratio in variance_ratios:
        # Perform PCA on the combined feature matrix
        pca = PCA(variance_ratio)
        xtrain_pca = pca.fit_transform(xtrain_combined)
        xval_pca = pca.transform(xval_combined)
        print(xtrain_pca.shape)

        # Fit classifier
        mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=100, random_state=42)
        clf = OneVsRestClassifier(mlp)
        clf.fit(xtrain_pca, ytrain)

        # Validation Predictions
        y_pred_val_prob = clf.predict_proba(xval_pca)
        y_pred_val = (y_pred_val_prob >= t).astype(int)

        # Calculate Micro F1 score
        f1_macro = f1_score(yval, y_pred_val, average="macro")

        # Print and update best parameters
        print(f"t: {t}, Variance Ratio: {variance_ratio}, Micro F1 score validation: {f1_macro}")
        if f1_macro > best_f1_macro:
            best_f1_macro = f1_macro
            best_params['t'] = t
            best_params['variance_ratio'] = variance_ratio

# Print best parameters
print("Best Parameters:")
print(best_params)


(2429, 44)


/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptro

t: 0.05, Variance Ratio: 0.9, Micro F1 score validation: 0.19640017386843664
(2429, 103)


/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptro

t: 0.05, Variance Ratio: 0.95, Micro F1 score validation: 0.20431017086661635
(2429, 365)


...

t: 0.5, Variance Ratio: 0.9, Micro F1 score validation: 0.10677625242620813
(2429, 103)


/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptro

t: 0.5, Variance Ratio: 0.95, Micro F1 score validation: 0.12440225051114752
(2429, 365)


/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptro

t: 0.5, Variance Ratio: 0.99, Micro F1 score validation: 0.11874921043520692
Best Parameters:
{'t': 0.05, 'variance_ratio': 0.95}


## Fitting the best model on full data

In [14]:
xtrain,ytrain=X,y

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Define TF-IDF vectorizer with additional parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200, norm='l2', sublinear_tf=True,
                                   stop_words='english', ngram_range=(1, 2))

# Transform train, validation, and test sets
xtrain_tfidf = tfidf_vectorizer.fit_transform(xtrain[:, 0])
xtest_tfidf = tfidf_vectorizer.transform(test['cleaned_plot_synopsis'])


In [16]:
# Combining tf-idf and embeddings

xtrain_combined = np.hstack((xtrain_tfidf.toarray(), xtrain[:, 1:]))
xval_combined = np.hstack((xval_tfidf.toarray(),xval[:, 1:]))
xtest_combined = np.hstack((xtest_tfidf.toarray(), distilroberta_test))

print(xtrain_combined.shape)
print(ytrain.shape)
print(xtest_combined.shape)

(3000, 968)
(3000, 70)
(864, 968)


In [17]:
pca = PCA(0.95)
xtrain_pca = pca.fit_transform(xtrain_combined)

mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=100, random_state=42)
clf = OneVsRestClassifier(mlp)
clf.fit(xtrain_pca, ytrain)

/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptro

OneVsRestClassifier(estimator=MLPClassifier(max_iter=100, random_state=42))

In [18]:
xtest_pca = pca.transform(xtest_combined)
    
t=0.05
# Test Predictions
y_pred_test_prob = clf.predict_proba(xtest_pca)
y_pred_test = (y_pred_test_prob >= t).astype(int)

# y_pred_val_tags = mlb.inverse_transform(y_pred_val)
# print(y_pred_val_tags)
# y_pred_test_tags = mlb.inverse_transform(y_pred_test)
# print(y_pred_test_tags)

submission_df = pd.concat([test, pd.DataFrame(y_pred_test, columns=classes_list)], axis=1)
submission_df.drop(['plot_synopsis','cleaned_plot_synopsis'], axis=1, inplace=True)
submission_df.to_csv('my_submission_mlp_tfidf_distilroberta_pca_tuned.csv', index=False)
submission_df

,index,revenge,boring,depressing,brainwashing,historical_fiction,comic,psychological,neo_noir,plot_twist,...,sadist,western,adult_comedy,dramatic,suspenseful,insanity,prank,intrigue,autobiographical,grindhouse_film
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,3,1,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,4,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,859,1,0,0,0,0,0,0,1,0,...,1,0,0,0,1,0,0,0,0,0
860,860,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
861,861,1,1,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
862,862,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
